## Fetch the proposed datasets understudy

### Setup
    Install required libraries

In [ ]:
#!pip install jsonstat.py
import eurostat
import io
import pandas
import requests
import jsonstat

### Source 1: Eurostat data
    - Agriculture price indecies of product

In [7]:
# price indecies of product 2015 base year
price_idx_products_annual_code = 'apri_pi15_outa'
price_idx_products_annual_dataframe = eurostat.get_data_df(price_idx_products_annual_code, flags=False)
price_idx_products_annual_dataframe.sample(5)

,p_adj,unit,product,geo\time,2021,2020,2019,2018,2017,2016,2015
2954,NI,PCH_PRE,041100,UK,NaN,NaN,6.72,27.51,-19.76,28.99,NaN
6556,RI,I15,119000,IT,NaN,106.89,117.05,107.22,103.26,99.30,100.0
3288,NI,PCH_PRE,050000,HR,-13.26,-14.15,23.55,17.85,-5.14,12.83,NaN
8452,RI,PCH_PRE,090000,PT,NaN,-12.71,-7.51,17.65,-7.39,8.57,NaN
2247,NI,I15,141000,DK,108.05,105.61,105.28,97.71,104.23,94.13,100.0


In [8]:
# price indecies of product 2010 base year
price_idx_products_annual2010_code = 'apri_pi10_outa'
price_idx_products_annual2010_dataframe = eurostat.get_data_df(price_idx_products_annual2010_code, flags=False)
price_idx_products_annual2010_dataframe = (price_idx_products_annual2010_dataframe
                                           .rename(columns={price_idx_products_annual2010_dataframe
                                                            .columns[3]:'geotime'})
                                          )
price_idx_products_annual2010_dataframe.sample(5)



,p_adj,unit,product,geotime,2017,2016,2015,2014,2013,2012,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
5423,RI,I10,061910,LV,98.5,119.5,96.3,112.7,108.3,85.1,...,72.5,81.4,109.5,84.3,78.7,59.8,39.5,69.9,66.1,96.1
936,NI,I10,050000,IT,111.5,116.5,100.6,108.2,129.9,98.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3602,NI,PCH_PRE,090000,EL,-2.7,2.8,20.2,-3.0,-3.5,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,NI,I10,039300,SE,135.8,108.3,107.2,106.0,103.6,101.3,...,106.3,112.5,93.8,75.0,75.0,NaN,NaN,NaN,NaN,NaN
1444,NI,I10,070000,LU,112.8,112.7,110.4,110.4,110.4,110.4,...,99.1,99.1,98.1,96.9,95.1,93.7,99.2,96.0,94.5,93.1


In [9]:
price_idx_products_annual2010_dataframe["geotime"].value_counts()

EU28    452
ES      392
PT      384
HR      376
FR      364
EL      364
IT      356
HU      348
AT      348
CY      340
BG      336
RO      324
LV      296
UK      292
PL      276
BE      272
SI      272
SE      268
CZ      268
SK      268
DK      264
NL      240
DE      236
MT      232
LT      228
LU      220
FI      220
IE      168
Name: geotime, dtype: int64

In [11]:
price_idx_products_annual2010_dataframe_ie= price_idx_products_annual2010_dataframe.query("geotime=='IE'")

In [18]:
 x =( price_idx_products_annual2010_dataframe_ie
     .melt(
         id_vars = ["p_adj",
                    "unit",
                    "geotime",
                    "product"],
         var_name = "year",
         value_name="priceIDX")
    )
x.sample(5)

,p_adj,unit,geotime,product,year,priceIDX
2746,NI,PCH_PRE,IE,041960,2001,NaN
1399,NI,PCH_PRE,IE,041920,2009,0.0
883,NI,PCH_PRE,IE,011000,2012,16.4
2593,NI,PCH_PRE,IE,115000,2002,NaN
1804,RI,I10,IE,140000,2007,101.9


## CSO
## Agriculture Area Used and Crop Production
    AQA03 - Crop Yield (1985 - 2007)
    AQA04 - Crop Yield (2008 - 2020)

In [101]:
# Note of where the URL comes from: https://data.cso.ie/table/AQA03

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AQA03%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

crop_yield8507_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

#for key, value in crop_yield8507_df.iloc[0:,0:3].iteritems():
#    print(key, value.unique())
#    print("\n")

crop_yield8507_df = ( crop_yield8507_df
 .pivot_table( 
     columns = "Statistic"
     ,index = ['Year',
               'Type of Crop',
               'UNIT'
              ]
     ,values='VALUE'
     ,dropna = True
             )
 .reset_index()
   )
crop_yield8507_df.sample(5)

Statistic,Year,Type of Crop,UNIT,Area under Crops,Crop Yield,Crop Yield per Hectare
752,1999,Winter oats,Tonnes,NaN,NaN,7.8
585,1996,Sugar beet,000 Hectares,32.3,NaN,NaN
22,1985,Spring wheat,000 Tonnes,NaN,100.0,NaN
402,1992,Winter oats,000 Hectares,11.6,NaN,NaN
827,2001,Sugar beet,Tonnes,NaN,NaN,48.2


In [94]:
# Note of where the URL comes from: https://data.cso.ie/table/AQA04

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AQA04%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

crop_yield0820_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

# pivot
crop_yield0820_df =( crop_yield0820_df
 .pivot_table( 
     columns = "Statistic"
     ,index = ['Year',
               'Type of Crop',
               'UNIT'
              ]
     ,values='VALUE'
     ,dropna = True
             )
 .reset_index()
 .rename(columns={"Crop Production":"Crop Yield"})                   
   )
crop_yield0820_df

Statistic,Year,Type of Crop,UNIT,Area under Crops,Crop Yield,Crop Yield per Hectare
0,2008,Beans and peas,000 Hectares,2.5,NaN,NaN
1,2008,Beans and peas,000 Tonnes,NaN,11.1,NaN
2,2008,Beans and peas,Tonnes,NaN,NaN,4.5
3,2008,Oilseed rape,000 Hectares,5.6,NaN,NaN
4,2008,Oilseed rape,000 Tonnes,NaN,20.3,NaN
...,...,...,...,...,...,...
502,2020,Winter oats,000 Tonnes,NaN,67.6,NaN
503,2020,Winter oats,Tonnes,NaN,NaN,8.2
504,2020,Winter wheat,000 Hectares,35.4,NaN,NaN
505,2020,Winter wheat,000 Tonnes,NaN,307.6,NaN


In [102]:
# append crop yield from 1985 tp 2020 
crop_yield_ie_df = crop_yield8507_df.append(crop_yield0820_df)

# write to csv

## Agricultural Input and Output Price Indices
    1995 - 2010
    2005 - 2017
    2014 - 2020

In [103]:
# Note of where the URL comes from: https://data.cso.ie/table/AQA04

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AHA01%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

prc_idx_9510_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

for key, value in prc_idx_9510_df.iloc[0:,0:-1].iteritems():
    print(key, value.unique())
    print("\n")

print(prc_idx_9510_df.head())

,Statistic,Year,Agricultural Product,UNIT,VALUE
0,Agricultural Input and Output Price Indices,1995,Total Index (Output),Base 2000=100,110.3
1,Agricultural Input and Output Price Indices,1995,Total Livestock and Livestock Products (Output),Base 2000=100,110.2
2,Agricultural Input and Output Price Indices,1995,Total Livestock Products (Output),Base 2000=100,105.5
3,Agricultural Input and Output Price Indices,1995,Milk (Output),Base 2000=100,105.2
4,Agricultural Input and Output Price Indices,1995,Total Livestock (Output),Base 2000=100,113.8


Statistic ['Agricultural Input and Output Price Indices']


Year [1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008
 2009 2010]


Agricultural Product ['Total Index (Output)' 'Total Livestock and Livestock Products (Output)'
 'Total Livestock Products (Output)' 'Milk (Output)'
 'Total Livestock (Output)' 'Total Cattle (Output)'
 'Finished Cattle (Output)' 'Cows for Slaughter (Output)'
 'Store Cattle (Output)' 'Sheep (Output)' 'Pigs (Output)'
 'Poultry (Output)' 'Total Crop Products (Output)' 'Cereals (Output)'
 'Sugarbeet (Output)' 'Potatoes (Output)' 'Vegetables (Output)'
 'Total Index (Input)' 'All Feeding Stuffs'
 'Straight Feeding Stuffs (Input)'
 'Compound Feeding Stuffs - Calf (Input)'
 'Compound Feeding Stuffs - Cattle (Input)'
 'Compound Feeding Stuffs - Pig (Input)'
 'Compound Feeding Stuffs - Poultry (Input)' 'All Fertilisers  (Input)'
 'Straight Fertilisers  (Input)' 'Compound Fertilisers - N.P.K. (Input)'
 'Compound Fertilisers - P.K. (Input)' 'All Energ

In [108]:
# pivot
prc_idx_9510_df =( prc_idx_9510_df
 .pivot_table( 
     columns = "Agricultural Product"
     ,index = ['Year',
               'UNIT'
              ]
     ,values='VALUE'
     ,dropna = True
             )
 .reset_index()
   )
prc_idx_9510_df.sample(10)

Agricultural Product,Year,UNIT,All Energy (Input),All Feeding Stuffs,All Fertilisers (Input),Cereals (Output),Compound Feeding Stuffs - Calf (Input),Compound Feeding Stuffs - Cattle (Input),Compound Feeding Stuffs - Pig (Input),Compound Feeding Stuffs - Poultry (Input),...,Sugarbeet (Output),Total Cattle (Output),Total Crop Products (Output),Total Index (Input),Total Index (Output),Total Livestock (Output),Total Livestock Products (Output),Total Livestock and Livestock Products (Output),Vegetables (Output),Veterinary Expenses (Including A.I.) (Input)
0,1995,Base 2000=100,71.700,103.400,96.400,130.600,108.300,106.000,99.500,98.900,...,100.6,120.700,110.600,93.400,110.300,113.800,105.500,110.200,92.50,85.800
14,2009,Base 2000=100,146.097,130.854,184.967,94.623,128.246,131.970,123.411,134.729,...,NaN,114.396,124.317,142.218,102.970,114.235,78.523,99.429,130.94,130.889
13,2008,Base 2000=100,168.892,143.410,220.604,133.070,140.907,147.379,136.520,136.622,...,NaN,127.779,137.919,155.940,122.265,124.526,112.812,119.669,138.99,128.945
3,1998,Base 2000=100,75.900,98.600,91.300,99.000,100.800,98.300,97.700,99.800,...,98.6,93.400,104.700,93.000,98.000,94.000,101.100,97.000,97.80,94.700
10,2005,Base 2000=100,131.700,109.000,124.500,96.600,106.400,108.800,109.500,108.600,...,103.7,105.600,112.000,118.000,102.300,105.600,93.700,100.700,116.10,117.600
9,2004,Base 2000=100,112.600,110.800,115.100,100.900,110.100,111.700,109.800,109.700,...,103.8,103.200,104.400,113.100,101.800,105.500,95.600,101.400,110.70,115.800
11,2006,Base 2000=100,143.100,111.100,133.100,110.600,107.400,111.200,110.100,109.900,...,NaN,113.200,133.900,123.100,107.400,112.000,90.300,103.000,123.60,122.400
8,2003,Base 2000=100,102.900,106.500,113.000,109.000,106.800,107.700,105.500,106.300,...,103.8,93.600,116.000,108.800,99.600,97.700,95.700,96.900,110.00,114.700
5,2000,Base 2000=100,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,...,100.0,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.00,100.000
7,2002,Base 2000=100,97.100,106.500,110.500,91.800,105.800,107.800,106.200,105.400,...,103.8,94.400,110.400,106.200,100.000,99.000,97.000,98.200,114.90,109.400


In [110]:

# Note of where the URL comes from: https://data.cso.ie/table/AQA04

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AHA03%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

prc_idx_0517_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

#for key, value in prc_idx_0517_df.iloc[0:,0:-1].iteritems():
#    print(key, value.unique())
#    print("\n")

#print(prc_idx_0517_df.head())

# pivot
prc_idx_0517_df =( prc_idx_0517_df
 .pivot_table( 
     columns = "Agricultural Product"
     ,index = ['Year',
               'UNIT'
              ]
     ,values='VALUE'
     ,dropna = True
             )
 .reset_index()
   )
prc_idx_0517_df.sample(10)

Agricultural Product,Year,UNIT,Agricultural input price index,Agricultural output price index,Animal output,Animal products,Animals,Calves,Cattle,Cattle excluding calves,...,Plant protection products,Potatoes including seeds,Poultry,Seeds,Sheep,Straight feeding stuffs,Straight fertilisers,Vegetables,Veterinary expenses,Wool
0,2005,Base 2010 = 100,84.60,89.64,90.82,93.06,89.40,138.20,89.63,89.29,...,97.00,63.58,87.26,95.55,76.92,81.99,82.84,93.03,89.27,113.63
8,2013,Base 2010 = 100,119.67,131.64,130.87,127.13,133.33,123.14,138.79,139.00,...,98.95,158.44,116.35,128.09,104.78,139.97,130.49,105.92,100.31,160.18
6,2011,Base 2010 = 100,111.12,115.17,115.91,114.34,116.95,148.16,120.25,119.88,...,97.75,69.00,106.14,105.99,107.87,125.08,127.44,101.43,99.57,180.19
4,2009,Base 2010 = 100,101.01,90.84,89.77,77.97,97.56,106.15,98.23,98.12,...,99.76,99.84,100.02,107.60,85.28,101.52,108.69,99.56,99.89,59.94
11,2016,Base 2010 = 100,108.58,110.42,110.28,88.00,124.96,150.12,129.75,129.48,...,99.92,116.13,112.15,115.74,109.01,103.55,103.01,104.56,105.54,146.25
9,2014,Base 2010 = 100,114.62,120.66,122.50,121.37,123.25,126.27,125.89,125.88,...,99.11,77.27,114.04,119.92,107.41,115.47,125.43,102.39,102.46,168.74
12,2017,Base 2010 = 100,107.84,121.23,122.75,115.35,127.63,146.07,131.76,131.57,...,101.63,102.74,111.53,115.64,108.69,105.94,97.34,104.33,106.10,94.15
2,2007,Base 2010 = 100,95.07,102.07,101.30,111.22,94.76,106.16,95.00,94.86,...,95.48,104.63,92.65,97.68,81.20,113.81,92.73,94.12,93.72,89.84
10,2015,Base 2010 = 100,112.15,116.00,117.08,95.64,131.22,166.19,139.09,138.73,...,98.45,100.30,112.18,116.43,108.68,108.35,125.89,103.41,103.73,209.86
1,2006,Base 2010 = 100,87.53,93.03,93.12,89.72,95.28,124.63,96.66,96.47,...,96.12,99.92,86.45,95.80,78.73,88.17,89.10,93.00,92.19,94.48


In [112]:

# Note of where the URL comes from: https://data.cso.ie/table/AQA04

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AHA04%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

prc_idx_1420_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

#for key, value in prc_idx_1420_df.iloc[0:,0:-1].iteritems():
#    print(key, value.unique())
#    print("\n")

#print(prc_idx_0517_df.head())

# pivot
prc_idx_1420_df =( prc_idx_1420_df
 .pivot_table( 
     columns = "Agricultural Product"
     ,index = ['Year',
               'UNIT'
              ]
     ,values='VALUE'
     ,dropna = True
             )
 .reset_index()
   )
prc_idx_1420_df.head()

Agricultural Product,Year,UNIT,Agricultural input price index,Agricultural output price index,Animal output,Animal products,Animals,Calves,Cattle,Cattle excluding calves,...,Pigs,Plant protection products,Potatoes including seeds,Poultry,Seeds,Sheep,Straight feeding stuffs,Straight fertilisers,Veterinary expenses,Wool
0,2014,Base 2015=100,103.60,106.84,107.75,127.67,94.54,76.31,90.37,90.45,...,109.41,100.04,87.48,101.20,104.24,98.83,107.71,99.47,99.15,81.48
1,2015,Base 2015=100,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
2,2016,Base 2015=100,97.85,95.12,93.68,91.13,95.37,90.48,93.02,93.03,...,102.60,102.28,128.50,99.48,98.46,99.92,97.12,82.26,101.94,69.67
3,2017,Base 2015=100,98.22,106.46,106.76,120.32,97.78,87.94,94.74,94.77,...,110.35,103.09,117.25,99.21,98.54,99.65,100.51,77.93,102.12,44.83
4,2018,Base 2015=100,102.74,104.42,102.44,113.54,95.08,76.17,93.36,93.46,...,96.48,103.93,140.44,99.50,98.16,105.06,111.80,84.09,105.29,42.81


## Value at Current Prices for Output, Input and Income in Agriculture

In [134]:

# Note of where the URL comes from: https://data.cso.ie/table/AEA01

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AEA01%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

prc_9021df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")

#for key, value in prc_idx_1420_df.iloc[0:,0:-1].iteritems():
#    print(key, value.unique())
#    print("\n")

#print(prc_idx_0517_df.head())

prc_9021df

,Statistic,Year,State,UNIT,VALUE
0,All Livestock,1990,State,Euro Million,2201.9
1,All Livestock,1991,State,Euro Million,2114.0
2,All Livestock,1992,State,Euro Million,2262.5
3,All Livestock,1993,State,Euro Million,2449.6
4,All Livestock,1994,State,Euro Million,2319.3
...,...,...,...,...,...
1723,Entrepreneurial Income,2017,State,Euro Million,3162.1
1724,Entrepreneurial Income,2018,State,Euro Million,2522.5
1725,Entrepreneurial Income,2019,State,Euro Million,2608.1
1726,Entrepreneurial Income,2020,State,Euro Million,2908.8


## Value at Current Prices for Subsidies on Products

In [132]:
# Note of where the URL comes from: https://data.cso.ie/table/AEA05

data_csv_url = "https://ws.cso.ie/public/api.jsonrpc?data=%7B%22jsonrpc%22:%222.0%22,%22method%22:%22PxStat.Data.Cube_API.ReadDataset%22,%22params%22:%7B%22class%22:%22query%22,%22id%22:%5B%5D,%22dimension%22:%7B%7D,%22extension%22:%7B%22pivot%22:null,%22codes%22:false,%22language%22:%7B%22code%22:%22en%22%7D,%22format%22:%7B%22type%22:%22CSV%22,%22version%22:%221.0%22%7D,%22matrix%22:%22AEA05%22%7D,%22version%22:%222.0%22%7D%7D"

response_json_rpc = requests.get(data_csv_url).json()

subsidies_df = pandas.read_csv(io.StringIO(response_json_rpc["result"]), sep=",")
subsidies_df


,Statistic,Year,State,UNIT,VALUE
0,All Subsidies on Products,1990,State,Euro Million,408.9
1,All Subsidies on Products,1991,State,Euro Million,357.3
2,All Subsidies on Products,1992,State,Euro Million,446.0
3,All Subsidies on Products,1993,State,Euro Million,466.4
4,All Subsidies on Products,1994,State,Euro Million,666.0
...,...,...,...,...,...
150,"Other Subsidies on Products (excluding Cattle,...",2016,State,Euro Million,0.0
151,"Other Subsidies on Products (excluding Cattle,...",2017,State,Euro Million,0.0
152,"Other Subsidies on Products (excluding Cattle,...",2018,State,Euro Million,0.0
153,"Other Subsidies on Products (excluding Cattle,...",2019,State,Euro Million,0.0
